# 달리진점
1. 데이터가 각 지사마다 시간순으로 이어져 있기때문에 지사와 지사의 경계에서도 시계열 데이터가 생기는 문제가 발생하여 지사별로 데이터프레임을 나눈후 각각 시계열 데이터화하여 다시 합침.
2. 데이터셋을 학습용과 테스트용으로 나누기 전에 시계열 형태로 변환하게 되면, 생성된 train 독립변수와 test 독립변수 간에 시계열 특성상 일부 구간이 겹칠 수 있다. 이러한 중복은 모델의 테스트 성능을 과대평가하는 문제로 이어질 수 있다. 따라서 계절에 따라 큰 변동을 보이는 양상까지 고려하여, 모델의 일반화 성능을 더욱 정확히 평가하기 위해 2023년에 해당하는 데이터를 전체 테스트셋으로 분리하여 사용하였습니다. 반대로 2021년 2022년에 해당하는 데이터는 학습 데이터셋으로 사용하였다.

In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [49]:
# 2021년 2022년은 학습 2023년은 테스트
df = pd.read_csv('./final_heat.csv')
le = LabelEncoder()
df['branch_id'] = le.fit_transform(df['branch_id'])
df['year'] = df['tm'].apply(lambda x : int(str(x)[:4]))

# 2023년 데이터
df_2023 = df[df['year'] == 2023].reset_index(drop=True).iloc[:,1:-1]

# 2021년, 2022년 데이터
df = df[df['year'] != 2023].iloc[:,1:-1]

In [50]:
df

,branch_id,hm,is_holiday,ta_chi,month_sin,month_cos,hour_sin,hour_cos,day_sin,day_cos,heat_demand
0,0,68.2,1,-8.2,0.0,1.000000,0.258819,0.965926,0.000000,1.00000,281.0
1,0,69.9,1,-8.6,0.0,1.000000,0.500000,0.866025,0.000000,1.00000,262.0
2,0,69.2,1,-8.8,0.0,1.000000,0.707107,0.707107,0.000000,1.00000,266.0
3,0,65.0,1,-8.9,0.0,1.000000,0.866025,0.500000,0.000000,1.00000,285.0
4,0,63.5,1,-9.2,0.0,1.000000,0.965926,0.258819,0.000000,1.00000,283.0
...,...,...,...,...,...,...,...,...,...,...,...
490536,18,61.1,1,-0.1,-0.5,0.866025,-0.965926,0.258819,-0.201299,0.97953,29.0
490537,18,68.5,1,-1.4,-0.5,0.866025,-0.866025,0.500000,-0.201299,0.97953,30.0
490538,18,75.8,1,-2.2,-0.5,0.866025,-0.707107,0.707107,-0.201299,0.97953,30.0
490539,18,76.9,1,-2.7,-0.5,0.866025,-0.500000,0.866025,-0.201299,0.97953,32.0


In [51]:
# 시계열 데이터셋 생성 함수
def create_dataset(data, time_step=48):    
    x_ = []  
    y_ = []
    b_id = []
    # 실습 ppt에 있는 시계열 데이터화 하는 함수는 대용량 데이터프레임(메타데이터 포함)은 너무 무겁고 오래 걸리지만
    # 데이터프레임을 넘파이 배열로 바꾸고 계산하면 훨씬 빠르고 가볍다.
    for i in range(len(data) - time_step - 1):        
        x = data[i:(i + time_step), 1:] # x에 정답 포함여부 설정하기
        y = data[(i + time_step), -1]
        b = data[i:(i + time_step), 0] 
        if np.isnan(x).any() or np.isnan(y).any():
            continue
        else :
            x_.append(x)
            y_.append(y)
            b_id.append(b)
    return np.array(x_), np.array(y_), np.array(b_id)

# 지사별로 데이터 나눈후 create_dataset으로 지사별로 시계열 데이터셋 생성하고 지사별로 나눴던거 다시 concat하는 함수
def split_x_y_bid(data):
    X_all = []
    Y_all = []
    B_all = []

    for i in range(len(data['branch_id'].unique())):
        globals()["df_"+str(i)] = data[data['branch_id'] == i ]
        print(f"df_{i}  : ", globals()[f"df_{i}"].shape[0],"개")
        X, Y, B_id = create_dataset(globals()[f"df_{i}"].values, time_step=48) # 24시간 단위로 자른다. -> 48시간전 데이터
        X_all.append(X)
        Y_all.append(Y)
        B_all.append(B_id)
        print(" 완료\n")
    
    X = np.concatenate(X_all, axis=0) # 넘파이 배열 concat
    Y = np.concatenate(Y_all, axis=0)
    B_id = np.concatenate(B_all, axis=0)
    return X, Y, B_id

In [114]:
# 2023년 데이터셋 생성
x_test, y_test, b_test = split_x_y_bid(df_2023)

print("결측치 수", np.isnan(x_test).sum() + np.isnan(y_test).sum() + np.isnan(b_test).sum())
print("X_2023 shape : ", x_test.shape)
print("Y_2023 shape : ", y_test.shape)
print("B_id_2023 shape : ", b_test.shape)

df_0  :  8760 개
 완료

df_1  :  8760 개
 완료

df_2  :  8760 개
 완료

df_3  :  8760 개
 완료

df_4  :  8760 개
 완료

df_5  :  8760 개
 완료

df_6  :  8760 개
 완료

df_7  :  8760 개
 완료

df_8  :  8760 개
 완료

df_9  :  8760 개
 완료

df_10  :  8760 개
 완료

df_11  :  8760 개
 완료

df_12  :  8760 개
 완료

df_13  :  8760 개
 완료

df_14  :  8760 개
 완료

df_15  :  8760 개
 완료

df_16  :  8760 개
 완료

df_17  :  8760 개
 완료

df_18  :  8760 개
 완료

결측치 수 0
X_2023 shape :  (158111, 48, 10)
Y_2023 shape :  (158111,)
B_id_2023 shape :  (158111, 48)


In [55]:
# 2021년,2022년 데이터셋 생성 및 데이터 섞기
X, Y, B_id = split_x_y_bid(df)
rng = np.random.default_rng(seed=1)  # 난수 생성기 객체 생성
indices = rng.permutation(len(x)) # 무작위 배열 생성
x_train = X[indices]
y_train = Y[indices]
b_train = B_id[indices]

print("결측치 수", np.isnan(X_train).sum() + np.isnan(Y_train).sum() + np.isnan(B_train).sum())
print("X shape : ", X_train.shape)
print("Y shape : ", Y_train.shape)
print("B_id shape : ", B_train.shape)

df_0  :  17519 개
 완료

df_1  :  17519 개
 완료

df_2  :  17519 개
 완료

df_3  :  17519 개
 완료

df_4  :  17519 개
 완료

df_5  :  17519 개
 완료

df_6  :  17519 개
 완료

df_7  :  17519 개
 완료

df_8  :  17519 개
 완료

df_9  :  17519 개
 완료

df_10  :  17519 개
 완료

df_11  :  17519 개
 완료

df_12  :  17519 개
 완료

df_13  :  17519 개
 완료

df_14  :  17519 개
 완료

df_15  :  17519 개
 완료

df_16  :  17519 개
 완료

df_17  :  17519 개
 완료

df_18  :  17519 개
 완료



NameError: name 'Y_train' is not defined

In [77]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import optuna

# 모델 하이퍼 파라미터 튜닝
def build_model(units_1, units_2):
    seq_input = Input(shape=(X.shape[1], X.shape[2]), name='sequence_input')
    branch_input = Input(shape=(B_id.shape[1],), name='branch_id_input') 
    branch_embed = Embedding(input_dim=19, output_dim=4)(branch_input)
    merged = Concatenate(axis=-1)([seq_input, branch_embed])
    
    x = LSTM(units_1, return_sequences=True)(merged)
    x = LSTM(units_2, return_sequences=False)(x)
    final_output = Dense(1)(x)

    model = Model(inputs=[seq_input, branch_input], outputs=final_output, name='lstm')
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def objective(trial):
    units_1 = trial.suggest_categorical('units_1', [128, 256])
    units_2 = trial.suggest_categorical('units_2', [128, 256])
    batch_size = trial.suggest_categorical('batch_size', [256, 512])

    kf = KFold(n_splits=4, shuffle=True, random_state=1)
    val_losses = []
    print(f"units_1 : {units_1}, units_2 : {units_2}, batch_size : {batch_size}")
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\n===== Fold {fold + 1} =====")
        b_train, b_val = B_id[train_idx] , B_id[val_idx]
        x_train, x_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx], Y[val_idx]

        model = build_model(units_1, units_2)

        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

        history = model.fit(
            [x_train, b_train], y_train,
            validation_data=([x_val, b_val], y_val),
            epochs=100,
            batch_size=batch_size,
            callbacks=[es],
            verbose=0
        )
        val_losses.append(min(history.history['val_loss']))

    return np.mean(val_losses)

# Optuna 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=8, show_progress_bar = True)

print("Best trial : ", study.best_trial.params)
print("Best loss : ", study.best_trial.values)

[I 2025-06-17 12:07:36,015] A new study created in memory with name: no-name-cd0683b4-9f7b-49c1-b935-468142a584bb


  0%|          | 0/8 [00:00<?, ?it/s]

units_1 : 128, units_2 : 256, batch_size : 256

===== Fold 1 =====
Epoch 44: early stopping
Restoring model weights from the end of the best epoch: 34.

===== Fold 2 =====
Epoch 27: early stopping
Restoring model weights from the end of the best epoch: 17.

===== Fold 3 =====
Epoch 58: early stopping
Restoring model weights from the end of the best epoch: 48.

===== Fold 4 =====
Epoch 54: early stopping
Restoring model weights from the end of the best epoch: 44.
[I 2025-06-17 12:20:04,627] Trial 0 finished with value: 64.05445384979248 and parameters: {'units_1': 128, 'units_2': 256, 'batch_size': 256}. Best is trial 0 with value: 64.05445384979248.
units_1 : 128, units_2 : 256, batch_size : 512

===== Fold 1 =====
Epoch 42: early stopping
Restoring model weights from the end of the best epoch: 32.

===== Fold 2 =====
Epoch 72: early stopping
Restoring model weights from the end of the best epoch: 62.

===== Fold 3 =====
Epoch 42: early stopping
Restoring model weights from the end of 

In [78]:
units_1 = study.best_trial.params.get('units_1')
units_2 = study.best_trial.params.get('units_2')
batch_size = study.best_trial.params.get('batch_size')

# Best trial :  {'units_1': 256, 'units_2': 256, 'batch_size': 512}
# Best loss :  [58.0773229598999]

In [99]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.optimizers import Adam

# (batch, time_steps, features)
seq_input = Input(shape=(X.shape[1], X.shape[2]), name='sequence_input')
# (batch, time_steps)
branch_input = Input(shape=(B_id.shape[1],), name='branch_id_input')  
# (batch, time_steps, 4)
branch_embed = Embedding(input_dim=19, output_dim=4)(branch_input)

# (batch, time_steps, features + 4)
merged = Concatenate(axis=-1)([seq_input, branch_embed])
x = LSTM(units_1, return_sequences=True)(merged)
x = LSTM(units_2, return_sequences=False)(x)
final_output = Dense(1)(x)

model = Model(inputs=[seq_input, branch_input], outputs=final_output, name='lstm')
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

Model: "lstm"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ branch_id_input     │ (None, 48)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequence_input      │ (None, 48, 10)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_57        │ (None, 48, 4)     │         76 │ branch_id_input[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_57      │ (None, 48, 14)    │          0 │ sequence_input[0… │
│ (Concatenate)       │                   │            │ embedding_57[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_114 (LSTM)     │ (None, 48, 256)   │    277,504 │ concatenate_57[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_115 (LSTM)     │ (None, 256)       │    525,312 │ lstm_114[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_57 (Dense)    │ (None, 1)         │        257 │ lstm_115[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 803,149 (3.06 MB)

 Trainable params: 803,149 (3.06 MB)

 Non-trainable params: 0 (0.00 B)

In [115]:
from sklearn.model_selection import train_test_split
x_val, x_test, y_val, y_test, b_val, b_test = train_test_split(x_test, y_test, b_test, test_size = 0.2, random_state = 1)

print("x_val shape : ", x_val.shape)
print("x_test shape : ", x_test.shape)
print("y_val shape : ", y_val.shape)
print("y_test shape : ", y_test.shape)
print("b_val shape : ", b_val.shape)
print("b_test shape : ", b_test.shape)

x_val shape :  (126488, 48, 10)
x_test shape :  (31623, 48, 10)
y_val shape :  (126488,)
y_test shape :  (31623,)
b_val shape :  (126488, 48)
b_test shape :  (31623, 48)


In [100]:
from keras.callbacks import EarlyStopping

es = EarlyStopping( monitor="val_loss", patience = 10, restore_best_weights=True)

model.fit(
    [x_train, b_train], y_train,
    validation_data=([x_val, b_val], y_val),
    epochs=100,
    batch_size=batch_size,
    callbacks=[es]
)

Epoch 1/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 15866.0303 - val_loss: 6822.6904
Epoch 2/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 7222.6729 - val_loss: 3581.4749
Epoch 3/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 4102.9082 - val_loss: 2010.5728
Epoch 4/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 2384.8394 - val_loss: 1167.9502
Epoch 5/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 1556.1556 - val_loss: 702.1907
Epoch 6/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 979.7728 - val_loss: 454.7315
Epoch 7/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 624.4266 - val_loss: 295.7190
Epoch 8/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 431.9833 - val_loss: 207.9751
Epoch 9/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 297.2740 - val_loss: 152.1541
Epoch 10/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 213.5227 - val_loss: 126.2532
Epoch 11/100
564/564 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 168.42

In [101]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
def adj_r2_score(y_test, pred, p=X.shape[1]):
    return 1-(1-r2_score(y_test, pred)) * (len(y_test)-1) / (len(y_test) - p - 1)

pred = model.predict([x_test, b_test])    
ad_r2 = adj_r2_score(y_test, pred)
mse = mean_squared_error(y_test.reshape(-1,1), pred)
mae = mean_absolute_error(y_test.reshape(-1,1), pred)
r2 = r2_score(y_test.reshape(-1,1), pred)

print("R2 : ", r2)
print("Adjusted R2 : ", ad_r2, '\n')
print("MSE : ", mse)
print("RMSE : ", np.sqrt(mse))
print("MAE : ", mae)

989/989 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
R2 :  0.9950061846046331
Adjusted R2 :  0.9949985928158519 

MSE :  60.92172868965152
RMSE :  7.805237260304874
MAE :  4.535790074132853


## GRU

In [104]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Embedding, Concatenate
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import optuna

# 모델 하이퍼 파라미터 튜닝
def build_model(gru_units_1, gru_units_2):
    seq_input = Input(shape=(X.shape[1], X.shape[2]), name='sequence_input')
    branch_input = Input(shape=(B_id.shape[1],), name='branch_id_input') 
    branch_embed = Embedding(input_dim=19, output_dim=4)(branch_input)
    merged = Concatenate(axis=-1)([seq_input, branch_embed])
    
    x = GRU(gru_units_1, return_sequences=True)(merged)
    x = GRU(gru_units_2, return_sequences=False)(x)
    final_output = Dense(1)(x)

    model = Model(inputs=[seq_input, branch_input], outputs=final_output, name='lstm')
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def objective(trial):
    gru_units_1 = trial.suggest_categorical('gru_units_1', [128, 256])
    gru_units_2 = trial.suggest_categorical('gru_units_2', [128, 256])
    gru_batch_size = trial.suggest_categorical('gru_batch_size', [256, 512])

    kf = KFold(n_splits=4, shuffle=True, random_state=1)
    val_losses = []
    print(f"gru_units_1 : {gru_units_1}, units_2 : {gru_units_2}, gru_batch_size : {gru_batch_size}")
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\n===== Fold {fold + 1} =====")
        b_train, b_val = B_id[train_idx] , B_id[val_idx]
        x_train, x_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx], Y[val_idx]

        model = build_model(gru_units_1, gru_units_2)

        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

        history = model.fit(
            [x_train, b_train], y_train,
            validation_data=([x_val, b_val], y_val),
            epochs=100,
            batch_size=batch_size,
            callbacks=[es],
            verbose=0
        )
        val_losses.append(min(history.history['val_loss']))

    return np.mean(val_losses)

# Optuna 실행
gru_study = optuna.create_study(direction='minimize')
gru_study.optimize(objective, n_trials=8, show_progress_bar = True)

print("Best trial : ", gru_study.best_trial.params)
print("Best loss : ", gru_study.best_trial.values)

[I 2025-06-17 15:05:37,266] A new study created in memory with name: no-name-5bf774a3-1311-4a67-96f1-8541968cc7ed


  0%|          | 0/8 [00:00<?, ?it/s]

gru_units_1 : 256, units_2 : 256, gru_batch_size : 512

===== Fold 1 =====
Epoch 54: early stopping
Restoring model weights from the end of the best epoch: 44.

===== Fold 2 =====
Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 37.

===== Fold 3 =====
Epoch 57: early stopping
Restoring model weights from the end of the best epoch: 47.

===== Fold 4 =====
Epoch 77: early stopping
Restoring model weights from the end of the best epoch: 67.
[I 2025-06-17 15:19:40,960] Trial 0 finished with value: 64.79286193847656 and parameters: {'gru_units_1': 256, 'gru_units_2': 256, 'gru_batch_size': 512}. Best is trial 0 with value: 64.79286193847656.
gru_units_1 : 128, units_2 : 256, gru_batch_size : 512

===== Fold 1 =====
Epoch 70: early stopping
Restoring model weights from the end of the best epoch: 60.

===== Fold 2 =====
Epoch 51: early stopping
Restoring model weights from the end of the best epoch: 41.

===== Fold 3 =====
Epoch 81: early stopping
Restoring mo

In [119]:
print("Best trial : ", gru_study.best_trial.params)
print("Best loss : ", gru_study.best_trial.values)

Best trial :  {'gru_units_1': 128, 'gru_units_2': 256, 'gru_batch_size': 256}
Best loss :  [62.04658222198486]


In [110]:
gru_units_1 = gru_study.best_trial.params.get('gru_units_1')
gru_units_2 = gru_study.best_trial.params.get('gru_units_2')
gru_batch_size = gru_study.best_trial.params.get('gru_batch_size')

In [116]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam

seq_input = Input(shape=(X.shape[1], X.shape[2]), name='sequence_input')  
branch_input = Input(shape=(B_id.shape[1],), name='branch_id_input')      
branch_embed = Embedding(input_dim=19, output_dim=4)(branch_input)        

merged = Concatenate(axis=-1)([seq_input, branch_embed])                 
x = GRU(gru_units_1, return_sequences=True)(merged)
x = GRU(gru_units_2, return_sequences=False)(x)
final_output = Dense(1)(x)

model = Model(inputs=[seq_input, branch_input], outputs=final_output, name='gru')
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()

Model: "gru"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ branch_id_input     │ (None, 48)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequence_input      │ (None, 48, 10)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_104       │ (None, 48, 4)     │         76 │ branch_id_input[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_104     │ (None, 48, 14)    │          0 │ sequence_input[0… │
│ (Concatenate)       │                   │            │ embedding_104[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_84 (GRU)        │ (None, 48, 128)   │     55,296 │ concatenate_104[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_85 (GRU)        │ (None, 256)       │    296,448 │ gru_84[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_100 (Dense)   │ (None, 1)         │        257 │ gru_85[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 352,077 (1.34 MB)

 Trainable params: 352,077 (1.34 MB)

 Non-trainable params: 0 (0.00 B)

In [117]:
from keras.callbacks import EarlyStopping

es = EarlyStopping( monitor="val_loss", patience = 10, restore_best_weights=True)

model.fit(
    [x_train, b_train], y_train,
    validation_data=([x_val, b_val], y_val),
    epochs=100,
    batch_size=gru_batch_size,
    callbacks=[es]
)

Epoch 1/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 13542.6064 - val_loss: 3626.6848
Epoch 2/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3723.8247 - val_loss: 1194.5446
Epoch 3/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1390.0104 - val_loss: 450.5083
Epoch 4/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 591.1649 - val_loss: 220.3590
Epoch 5/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 283.4827 - val_loss: 143.3211
Epoch 6/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 169.7278 - val_loss: 97.8200
Epoch 7/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 119.8439 - val_loss: 92.9501
Epoch 8/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 100.8282 - val_loss: 79.9255
Epoch 9/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 94.3470 - val_loss: 76.2921
Epoch 10/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 86.4268 - val_loss: 74.4917
Epoch 11/100
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 85.247

In [118]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
def adj_r2_score(y_test, pred, p=X.shape[1]):
    return 1-(1-r2_score(y_test, pred)) * (len(y_test)-1) / (len(y_test) - p - 1)

pred = model.predict([x_test, b_test])    
ad_r2 = adj_r2_score(y_test, pred)
mse = mean_squared_error(y_test.reshape(-1,1), pred)
mae = mean_absolute_error(y_test.reshape(-1,1), pred)
r2 = r2_score(y_test.reshape(-1,1), pred)

print("R2 : ", r2)
print("Adjusted R2 : ", ad_r2, '\n')
print("MSE : ", mse)
print("RMSE : ", np.sqrt(mse))
print("MAE : ", mae)

989/989 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
R2 :  0.993922872068303
Adjusted R2 :  0.9939136333864533 

MSE :  74.13753007582797
RMSE :  8.610315329639674
MAE :  4.986189179299388


# 신경망 모델

In [56]:
df = pd.read_csv('./final_heat.csv')
le = LabelEncoder()
df['branch_id'] = le.fit_transform(df['branch_id'])

df.dropna(inplace=True)
df['year'] = df['tm'].apply(lambda x : int(str(x)[:4]))

df_val = df[df['year']==2023].reset_index(drop=True)
df_train = df[df['year']!=2023]

# 상관관계 높은 변수
df_train = df_train[['branch_id', 'is_holiday', 'hm', 'ta_chi', 'month_poly', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'heat_demand']]
df_val = df_val[['branch_id', 'is_holiday', 'hm', 'ta_chi', 'month_poly', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'heat_demand']]

# 기본변수
# df_train = df_train[['branch_id', 'is_holiday', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1',
#                      'hm', 'ta_chi', 'month_poly', 'hour_sin', 'hour_cos', 'day_sin',
#                      'day_cos', 'heat_demand']]
# df_val = df_val[['branch_id', 'is_holiday', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1',
#                      'hm', 'ta_chi', 'month_poly', 'hour_sin', 'hour_cos', 'day_sin',
#                      'day_cos', 'heat_demand']]

In [57]:
x_train = df_train.iloc[:,1:-1]
y_train = df_train.iloc[:,-1]
b_train = df_train[['branch_id']]

In [58]:
df_val_x = df_val.iloc[:,1:-1]
df_val_y = df_val.iloc[:,-1]
df_val_bid = df_val[['branch_id']]

In [59]:
x_val, x_test, y_val, y_test, b_val, b_test = train_test_split(df_val_x, df_val_y, df_val_bid, test_size = 0.05)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(149582, 8)
(149582,)
(7873, 8)
(7873,)


In [61]:
ss_x = StandardScaler()
ss_y = StandardScaler()

# 상관관계 높은 변수
feature = ["hm", "ta_chi", "month_poly"]
# 기본변수
# feature = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'ta_chi', 'month_poly']


x_train[feature] = ss_x.fit_transform(x_train[feature])
x_val[feature] = ss_x.transform(x_val[feature])
x_test[feature] = ss_x.transform(x_test[feature])

y_train = ss_y.fit_transform(np.array(y_train).reshape(-1,1))
y_val = ss_y.transform(np.array(y_val).reshape(-1,1))

In [62]:
print("X_trian shape : ", x_train.shape)
print("Y_trian shape : ", y_train.shape)

print("X_val shape : ", x_val.shape)
print("Y_val shape : ", y_val.shape)

print("X_test shape : ", x_test.shape)
print("Y_test shape : ", y_test.shape)

print("B_train shape : ", b_train.shape)
print("B_val shape : ", b_val.shape)
print("B_test shape : ", b_test.shape)

X_trian shape :  (290069, 8)
Y_trian shape :  (290069, 1)
X_val shape :  (149582, 8)
Y_val shape :  (149582, 1)
X_test shape :  (7873, 8)
Y_test shape :  (7873,)
B_train shape :  (290069, 1)
B_val shape :  (149582, 1)
B_test shape :  (7873, 1)


In [75]:
branch_embed.shape

(None, 4)

In [94]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, Flatten, Concatenate, BatchNormalization, LeakyReLU

input_ = Input(shape=(x_train.shape[1],), name='input')

branch_input = Input(shape=(1,), name='branch_input')  # (batch_size, 1)
branch_embed = Embedding(input_dim=19, output_dim=4)(branch_input)  # 19개 지사, 임베딩 차원 4
branch_embed = Flatten()(branch_embed)  # (batch_size, 4)

merged = Concatenate()([input_, branch_embed])

x = Dense(128, kernel_initializer="he_normal")(merged)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)
# x = Dropout(0.1)(x)

x = Dense(128, kernel_initializer="he_normal")(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)
# x = Dropout(0.1)(x)

# x = Dense(128, kernel_initializer="he_normal")(x)
# x = BatchNormalization()(x)
# x = LeakyReLU(negative_slope=0.1)(x)

output = Dense(1)(x)

model = Model(inputs=[input_, branch_input], outputs=output, name='MLP')
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

Model: "MLP"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ branch_input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_21        │ (None, 1, 4)      │         76 │ branch_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input (InputLayer)  │ (None, 8)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 4)         │          0 │ embedding_21[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_21      │ (None, 12)        │          0 │ input[0][0],      │
│ (Concatenate)       │                   │            │ flatten_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_52 (Dense)    │ (None, 128)       │      1,664 │ concatenate_21[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_52[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_32      │ (None, 128)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_53 (Dense)    │ (None, 128)       │     16,512 │ leaky_re_lu_32[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_53[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_33      │ (None, 128)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_54 (Dense)    │ (None, 1)         │        129 │ leaky_re_lu_33[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,405 (75.80 KB)

 Trainable params: 18,893 (73.80 KB)

 Non-trainable params: 512 (2.00 KB)

In [95]:
es = EarlyStopping(monitor="val_loss", patience = 10, restore_best_weights=True) 

# 그리드 서치 추가

model.fit(
    [x_train, b_train], 
    y_train,
    validation_data=([x_val, b_val], y_val),
    epochs=50,
    batch_size=512,
    callbacks = [es]
)

Epoch 1/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2713 - val_loss: 0.0485
Epoch 2/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 0.0394 - val_loss: 0.0380
Epoch 3/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - loss: 0.0332 - val_loss: 0.0400
Epoch 4/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - loss: 0.0328 - val_loss: 0.0438
Epoch 5/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - loss: 0.0311 - val_loss: 0.0395
Epoch 6/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - loss: 0.0290 - val_loss: 0.0338
Epoch 7/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - loss: 0.0285 - val_loss: 0.0404
Epoch 8/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: 0.0276 - val_loss: 0.0356
Epoch 9/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 0.0269 - val_loss: 0.0320
Epoch 10/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - loss: 0.0263 - val_loss: 0.0324
Epoch 11/50
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: 0.0266 - val_loss: 0.0321
Epoch 12/50
567/567 ━━━━━━━━━━━━

In [96]:
def adj_r2_score(y_true, y_pred, p=x_train.shape[1]):
    return 1-(1-r2_score(y_true, y_pred)) * (len(y_true)-1) / (len(y_true) - p - 1)

pred = model.predict([x_test, b_test])
pred_ = ss_y.inverse_transform(pred)
ad_r2 = adj_r2_score(y_test, pred_)
mse = mean_squared_error(y_test, pred_)
r2 = r2_score(y_test, pred_)

print("R2 : ", r2)
print("adjusted R2 : ", ad_r2)
print("RMSE : ", np.sqrt(mse))

# 기본변수
# R2 :  0.9698835280330486
# adjusted R2 :  0.9698337107362461
# RMSE :  18.884774654730865

247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step
R2 :  0.9625906783208432
adjusted R2 :  0.9625526220424311
RMSE :  21.724237939806677


In [240]:
model.save('neural_model1.h5')

In [ ]:
Convolution - Batch Normalization - Activation - Dropout - Pooling